In [34]:
from bs4 import BeautifulSoup
import json
import requests
import lxml.html as lh
import pandas as pd
import re

In [35]:
def clean(txt):
    return re.sub(
        r"[\s\r\n]",
        "",
        txt
    )

In [36]:
NBSP = u'\xa0'
BOROS = [
    'New York',
    'Bronx',
    'Kings',
    'Queens',
    'Richmond'
]

In [37]:
def get_url_boro(boro_num):
    return 'https://web.enrboenyc.us/CD22509AD'+str(boro_num)+'.html'

In [38]:
def get_url_ad_boro(ad_num, boro_num):
    return 'https://web.enrboenyc.us/CD22509AD'+str(ad_num)+str(boro_num)+'.html'

In [39]:
def get_table(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    return doc.xpath('//table')[2]

In [40]:
def get_ads_in_boro(table):
    ads_in_boro = []
    for tr in table:
        txt = clean(tr[0].text_content())
        if txt != NBSP:
            ads_in_boro.append(txt)
    return ads_in_boro[2:-1]

In [45]:
def get_all_ads_in_boros():
    city = []
    for (i,v) in enumerate(BOROS):
        table = get_table(get_url_boro(i+1))
        ads_in_boro = get_ads_in_boro(table)
        city.append(ads_in_boro)
    return city

In [42]:
def get_eds_in_ad(table, ad_name, boro_name):
    cands = []
    ad = []
    
    for td in table[0][2:-1]:
        cand = td.text_content()
        if cand != NBSP:
            cands.append(cand)


    for tr in table[2:-1]:
        ed_row = []
        for td in tr:
            txt = clean(td.text_content())
            if txt != NBSP and txt != '':
                ed_row.append(txt)
        ed = ed_row[0] ## get ED number
        for (i, result) in enumerate(ed_row[2:]): ## skip % reported column
            ad.append([boro_name, ad_name, ed, cands[i], int(result.replace('-','0'))])
            
    return ad

In [43]:
def get_all_eds_in_ads_in_boros():
    all_eds = []
    city = get_all_ads_in_boros()
    for (i, ads_in_boro) in enumerate(city):
#         print('starting boro: ', i+1)
        for ad in ads_in_boro:
#             print('scraping ad:', ad)
            table = get_table(get_url_ad_boro(ad[2:4], i+1))
            eds_in_ad = get_eds_in_ad(table, ad, BOROS[i])
            all_eds = all_eds + eds_in_ad
    return all_eds

In [44]:
raw_data = get_all_eds_in_ads_in_boros()

TypeError: can only concatenate tuple (not "int") to tuple

In [ ]:
data = pd.DataFrame(raw_data, columns=['Borough','AD','ED','Name','Votes'])
data.to_csv('./data.csv', index=False)